In [1]:
# Install required package (run this cell once)
!pip install pretty_midi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 45.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 4.8 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592287 sha256=c9b2b087f74cc22e9f0a2cdeecca0210e36f35bcff2ed20d340c23d65facb68d
  Stored in directory: /root/.cache/pip/wheels/e6/95/ac/15ceaeb2823b04d8e638fd1495357adb8d26c00ccac9d7782e
Successfully built pretty_midi


In [2]:
# Import required modules
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import pretty_midi
import os
from IPython.display import Audio, display

# ----- Step 1: Generate Synthetic Music Dataset -----

# Define the MIDI note range (for example, notes from Middle C (60) to C above (72))
vocab_start = 60
vocab_end = 72
vocab_size = vocab_end - vocab_start + 1  # e.g., 13 possible notes

num_sequences = 1000          # Number of sequences to generate
sequence_total_length = 50    # Total length of each generated sequence
input_sequence_length = 20    # Length of input sequences for training

# Generate synthetic sequences using a random walk so that adjacent notes are related.
sequences = []
for _ in range(num_sequences):
    seq = []
    # Start at a random note in the range
    note = np.random.randint(vocab_start, vocab_end + 1)
    seq.append(note)
    for _ in range(1, sequence_total_length):
        # Random step between -2 and +2 semitones
        step = np.random.choice([-2, -1, 0, 1, 2])
        note = note + step
        # Ensure the note stays within our defined range
        note = int(np.clip(note, vocab_start, vocab_end))
        seq.append(note)
    sequences.append(seq)

# Create training examples by sliding over each sequence:
X = []
y = []
for seq in sequences:
    # Convert note values to indices (0 to vocab_size-1)
    seq_indices = [note - vocab_start for note in seq]
    for i in range(len(seq_indices) - input_sequence_length):
        X.append(seq_indices[i:i+input_sequence_length])
        y.append(seq_indices[i+input_sequence_length])
X = np.array(X)
y = np.array(y)

print("Training examples:", X.shape, y.shape)

# ----- Step 2: Build and Train the LSTM Model -----

embedding_dim = 64
lstm_units = 128

model = Sequential([
    # The Embedding layer converts integer indices to dense vectors.
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=input_sequence_length),
    LSTM(lstm_units),
    Dense(vocab_size, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
model.summary()

# Train the model (adjust epochs as needed; training on a synthetic dataset is fast)
epochs = 50
batch_size = 64
model.fit(X, y, epochs=epochs, batch_size=batch_size)

# ----- Step 3: Generate New Music -----

def generate_music(seed_seq, gen_length=100, temperature=1.0):
    """
    Generates a new sequence of note indices.
    - seed_seq: a list of integers (length should equal input_sequence_length)
    - gen_length: number of notes to generate
    - temperature: controls randomness (higher => more random)
    """
    generated = seed_seq.copy()
    for _ in range(gen_length):
        # Prepare the input sequence (reshape to (1, sequence_length))
        input_seq = np.array(generated[-input_sequence_length:]).reshape(1, input_sequence_length)
        predictions = model.predict(input_seq, verbose=0)[0]
        # Apply temperature scaling
        predictions = np.log(predictions + 1e-8) / temperature
        exp_preds = np.exp(predictions)
        predictions = exp_preds / np.sum(exp_preds)
        # Sample the next note index from the probability distribution
        next_index = np.random.choice(range(vocab_size), p=predictions)
        generated.append(next_index)
    return generated

# Choose a random seed from the training set
seed_index = np.random.randint(0, len(X))
seed_sequence = list(X[seed_index])
print("Seed sequence (indices):", seed_sequence)

generated_sequence = generate_music(seed_sequence, gen_length=100, temperature=1.0)

# Convert indices back to MIDI note numbers
generated_notes = [n + vocab_start for n in generated_sequence]
print("Generated note sequence:", generated_notes)

# ----- Step 4: Convert Generated Notes to a MIDI File -----

def notes_to_midi(notes, output_file="generated.mid", note_duration=0.5, start_time=0.0):
    """
    Converts a list of MIDI note numbers into a MIDI file.
    - notes: list of MIDI note numbers (integers)
    - output_file: name/path for the output MIDI file
    - note_duration: duration (in seconds) for each note
    - start_time: starting time offset
    """
    pm = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=0)  # Acoustic Grand Piano
    current_time = start_time
    for note in notes:
        note_event = pretty_midi.Note(
            velocity=100,
            pitch=note,
            start=current_time,
            end=current_time + note_duration
        )
        instrument.notes.append(note_event)
        current_time += note_duration
    pm.instruments.append(instrument)
    pm.write(output_file)
    print(f"MIDI file saved as '{output_file}'")

# Convert the generated note sequence into a MIDI file
notes_to_midi(generated_notes, output_file="generated.mid")

# ----- (Optional) Listen to the MIDI File in Colab -----
# Note: Colab's Audio player may not render MIDI directly.
# For a better listening experience, you could convert the MIDI file to audio using a synthesizer.
# Below, we simply provide a link to the MIDI file.
if os.path.exists("generated.mid"):
    print("Generated MIDI file is available for download: 'generated.mid'")

Training examples: (30000, 20) (30000,)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 26s 45ms/step - loss: 1.9180
Epoch 2/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 38s 38ms/step - loss: 1.5096
Epoch 3/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 21s 39ms/step - loss: 1.4805
Epoch 4/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - loss: 1.4677
Epoch 5/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 20s 42ms/step - loss: 1.4663
Epoch 6/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 19s 38ms/step - loss: 1.4621
Epoch 7/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 22s 42ms/step - loss: 1.4608
Epoch 8/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 19s 38ms/step - loss: 1.4533
Epoch 9/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 18s 37ms/step - loss: 1.4527
Epoch 10/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 20s 42ms/step - loss: 1.4502
Epoch 11/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 18s 38ms/step - loss: 1.4436
Epoch 12/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 18s 38ms/step - loss: 1.4433
Epoch 13/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 19s 41ms/step - loss: 1.4456
Epoch 14/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 18s 39ms/step - loss: 1.4443
Epoch 15/50
469/469 ━━━━━━━━━